
<div class="jumbotron">
  <h1><i class="fa fa-bar-chart" aria-hidden="true"></i> Integracción y limpieza</h1>
  <p></p>
</div>

### Objetivo

Integrar todas las fuentes de datos en un único archivo con formato PICKLE, el cual sera utilizado en los pasos de análisis. El archivo se almacena en el folder `data/interim/`.

In [1]:
import pandas as pd
import  pickle
import numpy as np
%matplotlib auto
#cargar biblioteca de gráficos
import seaborn as sns
sns.set(style="ticks")
import pandas as pd
import glob
from matplotlib import pyplot as plt
import os
import urllib.request
from os import path

pd.set_option('display.width', 1000)


Using matplotlib backend: <object object at 0xffff62609360>


### Obtención de los datos 

In [2]:
# Lista de URs de los meses que se analizan 
urls = ['https://ecobici.cdmx.gob.mx/wp-content/uploads/2022/07/2021-08.csv',
        'https://ecobici.cdmx.gob.mx/wp-content/uploads/2022/07/2021-09.csv',
        'https://ecobici.cdmx.gob.mx/wp-content/uploads/2022/07/2021-10.csv']

# Download
for url in urls :
    filename = url.split('/')[-1]
    path_file = '../data/raw/' + filename
    if not path.exists(path_file):
        urllib.request.urlretrieve(url, path_file)

# Fuentes y diccionario de datos

## Sobre los datos

El dataset de este ejemplo ha sido obtenido de el portal de datos abiertos de Ecobici (https://ecobici.cdmx.gob.mx/datos-abiertos/) , un servicio de renta de bicicletas que ofrece el gobierno de la Ciudad de México. Tiene las siguientes características:

- Histórico de 2010 a 2023 de los cuales sólo se toma los meses Agosto, Septiembre y Octubre de año 2021
- Los eventos registrados son las disposiciones y retornos de bicicletas en cualquiera de las estaciones de servicio


## Diccionario de datos

* Genero_Usuario: Genero de usuario (M|F)
* Edad_Usuario: Edad del usuario
* Bici: ID de la bicicleta
* Ciclo_Estacion_Retiro: Número de las cicloestación de retiro
* Fecha_Retiro: Fecha en que la bicicleta comienza a ser utilizada por algún usuario de Ecobici (DD:MM:AAAA)
* Hora_Retiro: Hora en que la bicicleta comienza a ser utilizada por algún usuario de Ecobici (HH:mm:ss)
* Ciclo_EstacionArribo: Número de las cicloestación de arribo
* Fecha Arribo: Fecha en que el usuario vuelve a colocar la bicicleta en alguna cicloestación (DD:MM:AAAA)
* Hora_Arribo : Hora en que el usuario vuelve a colocar la bicicleta en alguna cicloestación (HH:mm:ss)

# ¿Qué transformaciones ha sufrido el archivo?

Los datos originalmente fueron descargados desde el website mencionado arriba, request por request, con un script de scrapping en python. Se descarcaron en formato CSV individuales y se agregarón al ambiente local de datos en el folder 'raw'

El folder 'raw'tambien almacena dos archivos que contien la ubicación de cada una de las estaciones de ECOBICI asi como su diccionario de datos respectivamente. Dichos archivos fueron obtenidos de https://datos.cdmx.gob.mx/dataset/infraestructura-vial-ciclista

- ecobici_cicloestaciones.xlsx
- mibici_descriptor_cartografia.xlsx



### Cargar datos

In [3]:
# Cargar todos los archivos
path = os.getcwd()
all_files = glob.glob(os.path.join(path , "../data/raw/*.csv"))

#lista de dataframes
li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)

In [4]:
df.head(10)

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_EstacionArribo,Fecha Arribo,Hora_Arribo
0,M,28,11302,73,31/07/2021,23:57:44,62.0,01/08/2021,00:00:08
1,M,33,10571,121,31/07/2021,23:54:00,143.0,01/08/2021,00:00:59
2,M,19,12451,132,31/07/2021,23:52:58,122.0,01/08/2021,00:01:24
3,M,32,8314,7,31/07/2021,23:23:41,7.0,01/08/2021,00:01:25
4,F,36,7993,7,31/07/2021,23:23:17,7.0,01/08/2021,00:01:32
5,M,34,10675,136,31/07/2021,23:56:42,68.0,01/08/2021,00:02:23
6,M,35,3922,465,31/07/2021,23:49:01,461.0,01/08/2021,00:02:45
7,M,35,10911,237,31/07/2021,23:29:25,24.0,01/08/2021,00:03:05
8,M,31,8901,237,31/07/2021,23:29:38,24.0,01/08/2021,00:03:12
9,F,37,6763,369,01/08/2021,00:00:25,366.0,01/08/2021,00:03:31


### Dimensiones del dataframe 

In [5]:
df.shape

(492101, 9)

<div class="alert alert-info" role="alert">
<ul>
<li> El dataset cuenta con 1,114,357 de ejemplos </ul>
</div>

### Features 

In [6]:
# Nombre de las columnas
df.columns

Index(['Genero_Usuario', 'Edad_Usuario', 'Bici', 'Ciclo_Estacion_Retiro', 'Fecha_Retiro', 'Hora_Retiro', 'Ciclo_EstacionArribo', 'Fecha Arribo', 'Hora_Arribo'], dtype='object')

### Cambiar el nombre de las variables

NA

### Valores nulos o faltantes

In [7]:
df.isnull().sum()

Genero_Usuario           10758
Edad_Usuario                 0
Bici                         0
Ciclo_Estacion_Retiro        0
Fecha_Retiro                 0
Hora_Retiro                  1
Ciclo_EstacionArribo         2
Fecha Arribo                 2
Hora_Arribo                  2
dtype: int64

### Valores nulos como porcentaje 

In [8]:
((df.isnull().sum() / len(df))*100).sort_values(ascending = False)

Genero_Usuario           2.186137
Ciclo_EstacionArribo     0.000406
Fecha Arribo             0.000406
Hora_Arribo              0.000406
Hora_Retiro              0.000203
Edad_Usuario             0.000000
Bici                     0.000000
Ciclo_Estacion_Retiro    0.000000
Fecha_Retiro             0.000000
dtype: float64

<div class="alert alert-info" role="alert">
<ul>
<li> En términos generales, se suelen considerar los siguientes grados de impacto, dependiendo del porcentaje de valores faltantes (dumb rules):

- Menos de 1%: Trivial (no relevante)
- 1-5%: Manejable
- 5-15%: Manejable mediante métodos sofisticados
- Más de 15%: Crítico, con impacto severo en cualquier tipo de interpretación
 
    
Debido a que la mayoría de la variables contiene datos ausentes con un porcentaje inferior al 1% se procederá a remover dichas muestras. La variables 'Genero_Usuario' al no rebasar el 5% de datos ausentes puede continuar dentro de las variables de análisis sin un mayor impacto que remover la muetras que contienen valores nulos.
    
<li> Nota: Por falta de tiempo no se lidia con los datos faltantes de la variable Genero_Usuario. En un entorno productivo si la varible sobre pasa en 1% se debe utilizar tecnicas de inputación de datos para reducir el porcentaje de valores nullos.
    
</div>

### Analisis de valores faltantes

In [9]:
# No aplica

### Inputación de valores faltantes

In [10]:
# Se eliminan
df.dropna(inplace=True)

In [11]:
df.shape

(481341, 9)

### Número de muestras eliminados


In [12]:

1114357-df.shape[0]

633016

### Transformar columnas a formato datetime

In [13]:
# Tipo de datos detectados por Pandas
df.dtypes


Genero_Usuario            object
Edad_Usuario               int64
Bici                       int64
Ciclo_Estacion_Retiro      int64
Fecha_Retiro              object
Hora_Retiro               object
Ciclo_EstacionArribo     float64
Fecha Arribo              object
Hora_Arribo               object
dtype: object

In [14]:
# Dar formato de datetime a las variables de tiempo y fechas 

df['Fecha Arribo'] = pd.to_datetime(df['Fecha Arribo'],format='%d/%m/%Y')
df['Fecha_Retiro'] = pd.to_datetime(df['Fecha_Retiro'],format='%d/%m/%Y')
df['Hora_Retiro_'] = pd.to_datetime(df['Hora_Retiro'],format= '%H:%M:%S',errors='ignore' ).dt.hour
df['Minuto_Retiro'] = pd.to_datetime(df['Hora_Retiro'],format= '%H:%M:%S',errors='ignore' ).dt.minute
df['Segundo_Retiro'] = pd.to_datetime(df['Hora_Retiro'],format= '%H:%M:%S',errors='ignore' ).dt.second


df['Hora_Arribo_'] = pd.to_datetime(df['Hora_Arribo'],format= '%H:%M:%S',errors='ignore' ).dt.hour
df['Minuto_Arribo'] = pd.to_datetime(df['Hora_Arribo'],format= '%H:%M:%S',errors='ignore' ).dt.minute
df['Segundo_Arribo'] = pd.to_datetime(df['Hora_Arribo'],format= '%H:%M:%S',errors='ignore' ).dt.second



In [15]:
# Ver tipo de datos 
df.dtypes


Genero_Usuario                   object
Edad_Usuario                      int64
Bici                              int64
Ciclo_Estacion_Retiro             int64
Fecha_Retiro             datetime64[ns]
Hora_Retiro                      object
Ciclo_EstacionArribo            float64
Fecha Arribo             datetime64[ns]
Hora_Arribo                      object
Hora_Retiro_                      int64
Minuto_Retiro                     int64
Segundo_Retiro                    int64
Hora_Arribo_                      int64
Minuto_Arribo                     int64
Segundo_Arribo                    int64
dtype: object

In [16]:
df

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_EstacionArribo,Fecha Arribo,Hora_Arribo,Hora_Retiro_,Minuto_Retiro,Segundo_Retiro,Hora_Arribo_,Minuto_Arribo,Segundo_Arribo
0,M,28,11302,73,2021-07-31,23:57:44,62.0,2021-08-01,00:00:08,23,57,44,0,0,8
1,M,33,10571,121,2021-07-31,23:54:00,143.0,2021-08-01,00:00:59,23,54,0,0,0,59
2,M,19,12451,132,2021-07-31,23:52:58,122.0,2021-08-01,00:01:24,23,52,58,0,1,24
3,M,32,8314,7,2021-07-31,23:23:41,7.0,2021-08-01,00:01:25,23,23,41,0,1,25
4,F,36,7993,7,2021-07-31,23:23:17,7.0,2021-08-01,00:01:32,23,23,17,0,1,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492095,M,67,10246,341,2021-10-06,15:54:28,321.0,2021-10-06,15:57:53,15,54,28,15,57,53
492096,M,24,1731,137,2021-10-06,15:50:07,121.0,2021-10-06,15:57:58,15,50,7,15,57,58
492097,M,24,3958,216,2021-10-06,15:51:22,217.0,2021-10-06,15:58:01,15,51,22,15,58,1
492098,M,37,9651,446,2021-10-06,15:49:23,407.0,2021-10-06,15:58:12,15,49,23,15,58,12


### Valores duplicados

In [17]:

# df.sort_values(['Fecha_Retiro','Bici','Hora_Retiro_'])

In [18]:
# Remover los datos duplicados
df.drop_duplicates(keep='first',inplace=True)

In [19]:
1077331- df.shape[0]

595990

<div class="alert alert-info" role="alert">
<ul><li> No se analizó los elementos duplicados debido a que en el dataset existe un número mínimo de estos (1)
</div>

### Verificar datos 

In [20]:
# Datos nulos
((df.isnull().sum() / len(df))*100).sort_values(ascending = False)

Genero_Usuario           0.0
Edad_Usuario             0.0
Bici                     0.0
Ciclo_Estacion_Retiro    0.0
Fecha_Retiro             0.0
Hora_Retiro              0.0
Ciclo_EstacionArribo     0.0
Fecha Arribo             0.0
Hora_Arribo              0.0
Hora_Retiro_             0.0
Minuto_Retiro            0.0
Segundo_Retiro           0.0
Hora_Arribo_             0.0
Minuto_Arribo            0.0
Segundo_Arribo           0.0
dtype: float64

### Obtener el nombre de las variables categorical

In [21]:
### No aplica

### Convertir las variables a tipo `category`

In [22]:
# NO aplica

### Almacenar el `dataframe` en formato pickle 

In [23]:
df

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_EstacionArribo,Fecha Arribo,Hora_Arribo,Hora_Retiro_,Minuto_Retiro,Segundo_Retiro,Hora_Arribo_,Minuto_Arribo,Segundo_Arribo
0,M,28,11302,73,2021-07-31,23:57:44,62.0,2021-08-01,00:00:08,23,57,44,0,0,8
1,M,33,10571,121,2021-07-31,23:54:00,143.0,2021-08-01,00:00:59,23,54,0,0,0,59
2,M,19,12451,132,2021-07-31,23:52:58,122.0,2021-08-01,00:01:24,23,52,58,0,1,24
3,M,32,8314,7,2021-07-31,23:23:41,7.0,2021-08-01,00:01:25,23,23,41,0,1,25
4,F,36,7993,7,2021-07-31,23:23:17,7.0,2021-08-01,00:01:32,23,23,17,0,1,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492095,M,67,10246,341,2021-10-06,15:54:28,321.0,2021-10-06,15:57:53,15,54,28,15,57,53
492096,M,24,1731,137,2021-10-06,15:50:07,121.0,2021-10-06,15:57:58,15,50,7,15,57,58
492097,M,24,3958,216,2021-10-06,15:51:22,217.0,2021-10-06,15:58:01,15,51,22,15,58,1
492098,M,37,9651,446,2021-10-06,15:49:23,407.0,2021-10-06,15:58:12,15,49,23,15,58,12


In [24]:
# Store data en formar pickle
with open('../data/interim/dataset_cleaning_level_1.pickle', 'wb') as handle:
    pickle.dump(df, handle)